# Pypowsybl dynawaltz simulation

In [ ]:
import logging
import pypowsybl as pp
import pypowsybl.dynamic as dyn
import pandas as pd

In [ ]:
network = pp.network.load("./dynawaltz/IEEE14.iidm")

You can add dynamic mappings with dataframes like this :

In [ ]:
df_alpha_beta = pd.DataFrame.from_dict({"static_id": [network.get_loads().loc[l].name for l in network.get_loads().index],
                             "parameter_set_id": ["LAB" for l in network.get_loads().index]})
df_GSTWPR = pd.DataFrame.from_dict({"static_id": [network.get_generators().loc[l].name for l in network.get_generators().index],
                             "parameter_set_id": ["GSTWPR" for l in network.get_generators().index]})

Don't forget to index the dataframe on the network element id column

In [ ]:
model_mapping = dyn.ModelMapping()
model_mapping.add_all_dynamic_mappings(dyn.DynamicMappingType.ALPHA_BETA_LOAD, df_alpha_beta.set_index("static_id"))
model_mapping.add_all_dynamic_mappings(dyn.DynamicMappingType.GENERATOR_SYNCHRONOUS_THREE_WINDINGS_PROPORTIONAL_REGULATIONS, df_GSTWPR.set_index("static_id"))

Adding events

In [ ]:
events = dyn.EventMapping()
events.add_event("EQD", dyn.EventType.BRANCH_DISCONNECTION, "_BUS____1-BUS____5-1_AC")

Adding curves, you can batch curves creation for a given id

In [ ]:
timeseries = dyn.CurveMapping()
timeseries.add_curves("_LOAD___2_EC", ["load_PPu", "load_QPu"])

Running the simulation (will load the default config file in ~/.itools folder)

In [ ]:
sim = dyn.Simulation()
res = sim.run(network, model_mapping, events, timeseries, 0, 30)

Display data of the run:

In [ ]:
res.status()
res.curves()